In [2]:
# Imports
import pandas as pd
import numpy as np
import random
import json
import ast

### Completed Tests

In [3]:
headers = ["name","health","armor_class","initiative","initiative_bonus","team","group"]
turn_track = pd.DataFrame(columns=headers)
turn_track

,name,health,armor_class,initiative,initiative_bonus,team,group


In [4]:
character = {
    "name":"Mr.",
    "health":56,
    "armor_class":20,
    "initiative": 15,
    "initiative_bonus":2,
    "team":"Heroes",
    "group":"Alone"
}
turn_track.append(character,ignore_index=True)


,name,health,armor_class,initiative,initiative_bonus,team,group
0,Mr.,56,20,15,2,Heroes,Alone


In [5]:
headers = ["name","health","armor_class"]
filter_track = pd.DataFrame(columns=headers)
character = {
    "name":"Mrs.",
    "health":568,
    "armor_class":200
}
filter_track = filter_track.append(character,ignore_index=True)
# filter_track
filter_track[filter_track.columns[[True,False,True]]]

,name,armor_class
0,Mrs.,200


In [6]:
party1 = pd.read_csv(".\Party1.csv")
party1["group"] = np.nan

In [7]:
def read_import(path,import_groups=True):
    party = pd.read_csv(path)
    if import_groups and ('group' in list(party)):
        return party
    else:
        party["group"] = np.nan
        return party
party2=read_import(".\Party2.csv")
party2.iloc[0]['group']

'Winterfell'

In [8]:
def initiative_based_group_assignment(groups:pd.DataFrame):
    df = groups.copy()
    df['total_initiative'] = df['initiative']+df['initiative_bonus']
    df.sort_values(by='total_initiative',ascending=False,inplace=True)
    df_count = {}
    team_order = df['team']
    group_placement = []
    lastTeam = None
    for team in team_order:
        if team != lastTeam :
            if team in df_count.keys() :
                df_count[team]+=1
            else :
                df_count[team]=1
            lastTeam = team
        group_placement.append(f"{team} {df_count[team]}")
    df['group']=group_placement
    return df.drop(columns="total_initiative")
party2_grouped = initiative_based_group_assignment(party2)
# party2_grouped

In [9]:
def roll(sided=20):
    return random.randint(1,sided)

def auto_initiative(groups:pd.DataFrame):
    df = groups.copy()
    df['initiative'] = df['initiative'].apply(lambda x: roll(20))
    return df
party2_random_init = auto_initiative(party2)
party2_random_init_grouped = initiative_based_group_assignment(party2_random_init)
# party2_random_init_grouped

In [10]:
def groups_list(turn_track:pd.DataFrame):
    return(turn_track['group'].unique())
# groups_list(party2_random_init_grouped)

In [11]:
def df_match_slice(df:pd.DataFrame,column,match):
    return df[df[column]==match]
group_to_split_df = df_match_slice(party2_random_init_grouped,'group','Stark 3')
# for member in group_to_split_df.index:
#     print(type(member))

In [12]:
party2_change_init = party2_random_init_grouped.copy()
character_selected = 'Eddard "Ned" Stark'
party2_change_init.loc[(party2_change_init['name'] == character_selected,'initiative')]=24
# party2_change_init

In [13]:
def df_set_slice(df:pd.DataFrame,column,match,new_data):
    df_slice = df[df[column] == match].copy()
    df_slice[column] = new_data
    df_copy = df.copy()
    df_copy.update(df_slice)
    return df_copy
# group_to_split_df = df_set_match_slice(party2_random_init_grouped,'group','Targaryen 1',['Test1','Test2'])
# group_to_split_df

In [14]:
type(group_to_split_df)

pandas.core.frame.DataFrame

In [15]:
def move_group(groups:pd.DataFrame,group_to_move,before_or_after,group_to_place):
    df = groups.copy()
    df.reset_index(drop=True,inplace=True)
    slice_group_to_move = df[df['group']==group_to_move].copy()
    df.drop(slice_group_to_move.index,inplace=True)
    df.reset_index(drop=True,inplace=True)
    slice_group_to_move.reset_index(drop=True,inplace=True)
    if before_or_after == "Before" :
        index_split_point = (df[df['group']==group_to_place].index[0]) #first index
    else :
        index_split_point = df[df['group']==group_to_place].index[-1]+1 #last index
    return pd.concat([df.iloc[:index_split_point],slice_group_to_move,df.iloc[index_split_point:]]).reset_index(drop=True)
move_test = party2_random_init_grouped.copy()
# move_test
move_group(move_test,"Stark 3","After","Targaryen 2")

,name,health,armor_class,initiative,initiative_bonus,team,group,attributes
0,Jorah Mormont,90,16,20,3,Targaryen,Targaryen 1,"{'weapon': ['dagger','sword'], 'feats': 'deadm..."
1,Sansa Stark,90,16,16,3,Stark,Stark 1,"{'weapon': ['dagger','sword'], 'feats': 'deadm..."
2,Jon Snow,90,16,15,3,Stark,Stark 1,"{'weapon': ['dagger','sword'], 'feats': 'deadm..."
3,Robert Baratheon,90,16,14,3,Stark,Stark 1,"{'weapon': ['dagger','sword'], 'feats': 'deadm..."
4,Viserys Targaryen,90,16,14,3,Targaryen,Targaryen 2,"{'weapon': ['dagger','sword'], 'feats': 'deadm..."
5,Catelyn Stark,90,16,9,3,Stark,Stark 3,"{'weapon': ['dagger'], 'feats': 'deadman'}"
6,Arya Stark,90,16,9,3,Stark,Stark 3,"{'weapon': ['dagger','needle']}"
7,Robb Stark,90,16,12,3,Stark,Stark 2,NaN
8,Jaime Lannister,90,16,10,3,Lannister,Lannister 1,"{'weapon': ['dagger','sword','metal hand']}"
9,Daenerys Targaryen,90,16,10,3,Targaryen,Targaryen 3,"{'weapon': ['dagger'], 'feats': 'deadman','dra..."


In [16]:
def move_character(groups:pd.DataFrame,person_to_move,destination_group):
    df = groups.copy()
    df.reset_index(drop=True,inplace=True)
    slice_character_to_move = df[df['name']==person_to_move].copy()
    slice_character_to_move['group']=destination_group
    df.drop(slice_character_to_move.index,inplace=True)
    index_split_point = df[df['group']==destination_group].index[-1]+1 #last index
    return pd.concat([df.iloc[:index_split_point],slice_character_to_move,df.iloc[index_split_point:]]).reset_index(drop=True)
move_char_test = party2_random_init_grouped.copy()
# move_test
# move_character(move_test,"Arya Stark","Stark 2")

In [17]:
def person_is_alone(groups:pd.DataFrame,person):
    group_name = groups[groups['name'] == person]['group'].values[0]
    return len(groups[groups['group']==group_name]) == 1

person_is_alone_test = party2_random_init_grouped.copy()
person_is_alone(person_is_alone_test,"Viserys Targaryen")

True

In [18]:
def multi_person_groups_list(groups):
    multi_person_groups_mask = [len(groups[groups['group']==x]) != 1 for x in groups['group'].unique()]
    multi_person_groups = groups['group'].unique()[multi_person_groups_mask]
    return multi_person_groups

multi_person_groups_list_test = party2_random_init_grouped.copy()
# multi_person_groups_list(multi_person_groups_list_test)

In [19]:
def move_character_to_new_group(groups:pd.DataFrame,person_to_move,new_group_name):
    if person_is_alone(groups,person_to_move) : return groups
    df = groups.copy()
    old_group=df.loc[(df["name"]==person_to_move,"group")].values[0]
    df.loc[(df["name"]==person_to_move,"group")]=new_group_name
    if df[df['group']==old_group].empty :
        return df # If character was the only member of a group, no need to rearrange
    return move_group(df,new_group_name,"After",old_group)
move_char_new_group_test = party2_random_init_grouped.copy()

move_character_to_new_group(move_char_new_group_test,"Cersei Lannister","Kinda Annoying")


,name,health,armor_class,initiative,initiative_bonus,team,group,attributes
6,Jorah Mormont,90,16,20,3,Targaryen,Targaryen 1,"{'weapon': ['dagger','sword'], 'feats': 'deadm..."
10,Sansa Stark,90,16,16,3,Stark,Stark 1,"{'weapon': ['dagger','sword'], 'feats': 'deadm..."
8,Jon Snow,90,16,15,3,Stark,Stark 1,"{'weapon': ['dagger','sword'], 'feats': 'deadm..."
1,Robert Baratheon,90,16,14,3,Stark,Stark 1,"{'weapon': ['dagger','sword'], 'feats': 'deadm..."
7,Viserys Targaryen,90,16,14,3,Targaryen,Targaryen 2,"{'weapon': ['dagger','sword'], 'feats': 'deadm..."
9,Robb Stark,90,16,12,3,Stark,Stark 2,NaN
2,Jaime Lannister,90,16,10,3,Lannister,Lannister 1,"{'weapon': ['dagger','sword','metal hand']}"
5,Daenerys Targaryen,90,16,10,3,Targaryen,Targaryen 3,"{'weapon': ['dagger'], 'feats': 'deadman','dra..."
3,Catelyn Stark,90,16,9,3,Stark,Stark 3,"{'weapon': ['dagger'], 'feats': 'deadman'}"
11,Arya Stark,90,16,9,3,Stark,Stark 3,"{'weapon': ['dagger','needle']}"


In [20]:
def merge_groups(groups:pd.DataFrame,merge_group_1,merge_group_2,merged_name):
    df = groups.copy()
    df = move_group(df,merge_group_1,"After",merge_group_2)
    df['group'].replace([merge_group_1,merge_group_2],[merged_name,merged_name],inplace=True)
    return df

def individual_groups(groups:pd.DataFrame):
    df = groups.copy()
    df['group']=df['name']
    return df

merge_test = party2_random_init_grouped.copy()
merge_test = individual_groups(merge_test)
# merge_test
# merge_groups(merge_test,'Eddard "Ned" Star',"Catelyn Stark","Stark Parents")

In [21]:
groups_list(party2)[groups_list(party2)!='Winterfell']

array(['Kings Landing', 'Nights Watch'], dtype=object)

In [22]:
def read_audit(path):
    audit_read = pd.read_csv(path)

    audit_tags = audit_read[audit_read['Audit Header'].str.contains('tags_')].reset_index(drop=True)
    audit_out = audit_read[audit_read['Audit Header'].str.contains('out_')].reset_index(drop=True)
    audit_meta = audit_read[audit_read['Audit Header'].str.contains('meta_')].reset_index(drop=True)
    
    audit_actions = audit_read.drop(index=audit_read[audit_read['Audit Header'].str.contains('tags_')].index)
    audit_actions = audit_actions.drop(index=audit_read[audit_read['Audit Header'].str.contains('out_')].index)
    audit_actions = audit_actions.drop(index=audit_read[audit_read['Audit Header'].str.contains('meta_')].index)

    audit_actions.reset_index(drop=True,inplace=True)

    audit_tags['Audit Header'] = audit_tags['Audit Header'].str[5:]
    audit_out['Audit Header'] = audit_out['Audit Header'].str[4:]
    audit_meta['Audit Header'] = audit_meta['Audit Header'].str[5:]

    return process_audit(audit_actions), process_audit(audit_out), process_audit(audit_tags), process_audit(audit_meta)

def process_audit(df:pd.DataFrame):
    df = df.set_index("Audit Header").transpose().reset_index(drop=True)
    audit_out = {}
    for col in df.columns : audit_out[col] = df[col].dropna().to_list()
    return audit_out

audit_actions,audit_outcome,audit_tags,audit_meta = read_audit("..\data\default_audit_actions.csv")

In [23]:
def next_turn(groups:pd.DataFrame,current_turn):
    groups.reset_index(drop=True,inplace=True)
    current_turns_last_index=groups[groups['group']==current_turn].index[-1]
    if current_turns_last_index == len(groups)-1 : #loop around detection
        return groups.iloc[0]['group']
    else:
        return groups.iloc[current_turns_last_index+1]['group']
# party2_random_init_grouped
# next_turn(party2_random_init_grouped,'Stark 4')

In [24]:
def groups_gathered_check(groups:pd.DataFrame):
    order_list = groups_list(groups)
    team_changes = groups["group"].shift() != groups["group"]
    return len(team_changes[team_changes==True])==len(order_list)
# groups_gathered_check(party2)

In [25]:
# party2_random_init_grouped[party2_random_init_grouped['group']=="Stark 2"].iloc[2]['attributes']

In [26]:
# di = {'test':[1,2,3,40],'diction':[5,6,7,8,"nine"],'number':4,'string':'hello'}
# party2_2=party2.copy()
# party2_2['attributes']=[di.copy() for _ in range(len(party2_2))]
# party2_2.to_csv(".\Party2_2.csv")

In [27]:
def attributes_list(groups:pd.DataFrame):
    df = groups[['name','attributes']].copy()
    df.dropna(inplace=True)
    all_attributes = []
    for index,character in df.iterrows():
        character_attributes = ast.literal_eval(character['attributes'])
        for key, values in character_attributes.items() :
            if type(values) == list :
                for item in values:
                    all_attributes.append(f"{character['name']} - {key} - {item}")
            else :
                all_attributes.append(f"{character['name']} - {key} - {values}")
    return all_attributes
# attributes_list(party2_random_init_grouped)

In [28]:
meta = audit_meta.copy()

def parse_meta_str(meta):
    meta_list = meta.strip('][').split(',')
    output = {}
    output["target"] = meta_list[1]
    output["modification"] = meta_list[2]
    output["wording"] = meta_list[0]
    return meta_list[0],output

def meta_to_dict(audit_meta,key='Results'):
    output = {}
    for meta in audit_meta[key]:
        parse_name, parse_dict = parse_meta_str(meta)
        output[parse_name] = parse_dict
    return output

def has_meta(result,meta_lookup:dict):
    return result in meta_lookup.keys()

# has_meta("Dealt Damage",meta_to_dict(meta))


In [29]:
def add_additional_info(result):
    return f'{result[0]} -> {result[2]} : {result[1]}'

result = ["info","This thing happened it was cool",["Jaime Lannister"]]
# add_info_test = party2_grouped.copy()
# add_additional_info(result)

In [30]:
def disrupt(turn_track,disruptor,disrupted_info):
    group_to_split = disrupted_info[0]
    group_name_1 = disrupted_info[1]
    split_decicions = disrupted_info[2]
    turn_track = df_set_slice(turn_track,"group",group_to_split,split_decicions)
    turn_track = move_character_to_new_group(turn_track, disruptor, disruptor)
    turn_track = move_group(turn_track,disruptor,"After",group_name_1)
    return turn_track

result = ["disrupt","Jon Snow",["Targaryen 1","Targaryen First",["Targaryen First","Targaryen First","Targaryen Second"]]]
disrupt_test = party2_grouped.copy()
# disrupt(disrupt_test,result[1],result[2])[['name','group']]

In [31]:
def add_audit(audit_trail:pd.DataFrame,turn,action_number,source,action,result,target,source_additional_info,target_additional_info,environment,damage,healing,additional_effects):
    audit_trail.loc[len(audit_trail.index)] = [
        turn, action_number,
        source,
        action,
        result,
        target,
        source_additional_info,
        target_additional_info,
        environment,
        damage,
        healing,
        additional_effects
    ]

def add_audit_note(audit_trail:pd.DataFrame,turn,action_number,note):
    add_audit(audit_trail,turn,action_number,"","","","","","","","","",note)
def add_audit_character_note(audit_trail:pd.DataFrame,turn,action_number,character,note):
    add_audit(audit_trail,turn,action_number,character,"","","","","","","","",note)

In [32]:
def adjust_health(turn_track,is_damage,numbers,target):
    health_mod = 1
    if is_damage == "-" : health_mod = -1
    for number in numbers :
        value = number[1]
        turn_track.loc[turn_track['name'].isin(target),'health'] += (health_mod * value)

health_test = party2_grouped.copy()
result = ['-', [['Arya Stark', 5], ['Sansa Stark', 4]], ['Jaime Lannister']]
adjust_health(health_test,result[0],result[1],result[2])
health_test

,name,health,armor_class,initiative,initiative_bonus,team,group,attributes
2,Jaime Lannister,81,16,14,3,Lannister,Lannister 1,"{'weapon': ['dagger','sword','metal hand']}"
1,Robert Baratheon,90,16,12,3,Stark,Stark 1,"{'weapon': ['dagger','sword'], 'feats': 'deadm..."
8,Jon Snow,90,16,10,3,Stark,Stark 1,"{'weapon': ['dagger','sword'], 'feats': 'deadm..."
9,Robb Stark,90,16,10,3,Stark,Stark 1,NaN
10,Sansa Stark,90,16,10,3,Stark,Stark 1,"{'weapon': ['dagger','sword'], 'feats': 'deadm..."
11,Arya Stark,90,16,10,3,Stark,Stark 1,"{'weapon': ['dagger','needle']}"
7,Viserys Targaryen,90,16,7,3,Targaryen,Targaryen 1,"{'weapon': ['dagger','sword'], 'feats': 'deadm..."
6,Jorah Mormont,90,16,6,3,Targaryen,Targaryen 1,"{'weapon': ['dagger','sword'], 'feats': 'deadm..."
5,Daenerys Targaryen,90,16,5,3,Targaryen,Targaryen 1,"{'weapon': ['dagger'], 'feats': 'deadman','dra..."
4,Cersei Lannister,90,16,4,3,Lannister,Lannister 2,"{'weapon': ['dagger','sword'], 'feats': 'deadm..."


In [33]:
def submit_action(turn_track,results_data,additional_log):
    for result in results_data:
        if result[0] in ['+','-'] : adjust_health(turn_track,result[0],result[1],result[2])
        elif result[0] in ['attribute','condition','info'] :
            if additional_log != "" : additional_log += '\n'
            additional_log += add_additional_info(result)
        elif result[0] == 'disrupt' : turn_track = disrupt(turn_track,result[1],result[2])
    return turn_track, additional_log

additional_log = ""
results_data = [
    ['-', [['Arya Stark', 5], ['Sansa Stark', 4]], ['Jaime Lannister']],
    ['-', [['Arya Stark', 5]], ['Jaime Lannister']],
    ["info","This thing happened it was cool",["Jaime Lannister"]],
    ["disrupt","Jon Snow",["Targaryen 1","Targaryen First",["Targaryen First","Targaryen First","Targaryen Second"]]]
    ]
submit_test = party2_grouped.copy()
# submit_test, additional_log = submit_action(submit_test,results_data,additional_log)
# print(additional_log)
# submit_test

# Tests

In [118]:
audit_headers = ["turn","action_number","source","action","result","target","source_additional_info","target_additional_info","environment","damage","healing","additional_effects"]
audit = pd.DataFrame(columns=audit_headers)
# add_audit(audit,
#     0,0, # turn, action_number
#     ['Sansa Stark','Arya Stark'], # source
#     ['Shoves', 'Attacks'], # action
#     ['Target Moves', 'Dealing Damage'], # result
#     ['Jaime Lannister'], # target
#     ['Arya Stark - weapon - needle'], # source_additional_info
#     ['Jaime Lannister - weapon - metal hand'], # target_additional_info
#     "Normal Conditions", # environment
#     [[[['Arya Stark', 5], ['Sansa Stark', 4]], ['Jaime Lannister']],[[['Arya Stark', 5]], ['Sansa Stark']]], # damage
#     [], # healing
#     ""# additional_effects
# )
# add_audit(audit,
#     0,1, # turn, action_number
#     ['Jon Snow', 'Robb Stark'], # source
#     ['Casts spell', 'Grapples'], # action
#     ['Dealing Damage', 'Healing'], # result
#     ['Robert Baratheon'], # target
#     ['Jon Snow - feats - deadman'], # source_additional_info
#     [], # target_additional_info
#     "Normal Conditions", # environment
#     [[[['Jon Snow', 10], ['Robb Stark', 8]], ['Jon Snow', 'Robb Stark']]], # damage
#     [[[['Jon Snow', 5], ['Robb Stark', 4]], ['Robert Baratheon']]], # healing
#     """attribute -> [] : ['Arya Stark - weapon - needle']]\n""" # additional_effects
# )
# add_audit(audit,
#     1,2, # turn, action_number
#     ['Viserys Targaryen', 'Daenerys Targaryen'],
#     ['Attacks'],
#     ['Inflicting Condition', 'Something Else Happened', 'Debuffing'],
#     ['Jon Snow'],
#     ['Viserys Targaryen - weapon - sword', 'Daenerys Targaryen - feats - deadman', 'Daenerys Targaryen - dragon - Dragon2'],
#     [],
#     "Normal Conditions",
#     [],
#     [],
#     """info -> ['Viserys Targaryen'] : Just made it crazy
# condition -> ['Jon Snow'] : ['Blinded']
# attribute -> ['Jon Snow'] : ['Daenerys Targaryen - dragon - Dragon2']"""
# )

add_audit_note(audit,0,0,"Groups created based on Team/Initiative")
add_audit(audit,0,0,['Jaime Lannister'], ['Moves'],[],"Moves behind tree",['Jaime Lannister - weapon - metal hand'],[],"Three-quarters cover",[],[],"")
add_audit(audit,
    0,1, # turn, action_number
    ['Jon Snow', 'Robb Stark'], # source
    ['Casts spell', 'Grapples'], # action
    ['Dealing Damage', 'Healing'], # result
    ['Robert Baratheon'], # target
    ['Jon Snow - feats - deadman'], # source_additional_info
    [], # target_additional_info
    "Normal Conditions", # environment
    [[[['Jon Snow', 10], ['Robb Stark', 8]], ['Jon Snow', 'Robb Stark']]], # damage
    [[[['Jon Snow', 5], ['Robb Stark', 4]], ['Robert Baratheon']]], # healing
    """attribute -> [] : ['Arya Stark - weapon - needle']]\n""" # additional_effects
)

def col_str_every_action(df,column_name,drop_cols):
    df_split = df.copy()
    df_split.drop(df_split[df_split[column_name]==''].index,inplace=True)
    df_split.drop(columns=drop_cols,inplace=True)
    df_split['additional_effects'] = df_split['additional_effects'].str.split('\n')
    df_split = df_split.explode('additional_effects').reset_index(drop=True)
    df_split.drop(df_split[df_split[column_name]==''].index,inplace=True)
    return df_split

def split_column_list(df,column_name,new_axis_names):
    df_split = df.copy()
    df_split.drop(df[df[column_name].isin(["",[]])].index,inplace=True)
    df_split[new_axis_names] = pd.DataFrame(df_split[column_name].to_list(),columns=new_axis_names,index=df_split.index)
    df_split.drop(columns=column_name,inplace=True)
    return df_split

def col_list_every_action(df,column_name,drop_cols):
    df_split = df.explode(column_name)
    df_split.dropna(inplace=True)
    df_split.drop(columns=drop_cols,inplace=True)
    df_split = split_column_list(df_split,column_name,['sources','target'])
    df_split = df_split.explode('sources')
    df_split = split_column_list(df_split,'sources',['source',column_name])
    return df_split

def audit_every_action_df(audit:pd.DataFrame):
    action_df = audit.copy()[["turn","action_number","damage","healing","additional_effects"]]
    concat_list = []
    if not all(action_df['damage'].isin(["",[]])) : concat_list.append(col_list_every_action(action_df,'damage',['healing','additional_effects']))
    if not all(action_df['healing'].isin(["",[]])) : concat_list.append(col_list_every_action(action_df,'healing',['damage','additional_effects']))
    if not all(action_df['additional_effects'].isin(["",[]])) : concat_list.append(col_str_every_action(action_df,'additional_effects',['damage','healing']))
    
    if len(concat_list) : return pd.concat(concat_list,ignore_index=True).sort_values(by=["turn","action_number"])
    return pd.DataFrame(columns=['turn','action_number','source','damage','healing','additional_effects'])

audit_every_action_df(audit)

,turn,action_number,target,source,damage,healing,additional_effects
4,0,0,NaN,NaN,NaN,NaN,Groups created based on Team/Initiative
0,0,1,"[Jon Snow, Robb Stark]",Jon Snow,10.0,NaN,NaN
1,0,1,"[Jon Snow, Robb Stark]",Robb Stark,8.0,NaN,NaN
2,0,1,[Robert Baratheon],Jon Snow,NaN,5.0,NaN
3,0,1,[Robert Baratheon],Robb Stark,NaN,4.0,NaN
5,0,1,NaN,NaN,NaN,NaN,attribute -> [] : ['Arya Stark - weapon - need...


In [122]:
party2_copy=party2.copy()
party2_copy.loc[party2['name']=='Jon Snow','group']='Jon Snow'
party2_copy

,name,health,armor_class,initiative,initiative_bonus,team,group,attributes
0,"Eddard ""Ned"" Stark",90,13,1,3,Stark,Winterfell,"{'weapon': ['dagger','sword'], 'feats': 'deadm..."
1,Robert Baratheon,90,16,12,3,Stark,Kings Landing,"{'weapon': ['dagger','sword'], 'feats': 'deadm..."
2,Jaime Lannister,90,16,14,3,Lannister,Kings Landing,"{'weapon': ['dagger','sword','metal hand']}"
3,Catelyn Stark,90,16,3,3,Stark,Winterfell,"{'weapon': ['dagger'], 'feats': 'deadman'}"
4,Cersei Lannister,90,16,4,3,Lannister,Kings Landing,"{'weapon': ['dagger','sword'], 'feats': 'deadm..."
5,Daenerys Targaryen,90,16,5,3,Targaryen,Nights Watch,"{'weapon': ['dagger'], 'feats': 'deadman','dra..."
6,Jorah Mormont,90,16,6,3,Targaryen,Nights Watch,"{'weapon': ['dagger','sword'], 'feats': 'deadm..."
7,Viserys Targaryen,90,16,7,3,Targaryen,Nights Watch,"{'weapon': ['dagger','sword'], 'feats': 'deadm..."
8,Jon Snow,90,16,10,3,Stark,Jon Snow,"{'weapon': ['dagger','sword'], 'feats': 'deadm..."
9,Robb Stark,90,16,10,3,Stark,Winterfell,NaN


# Possible Classes

In [35]:
class Character:
    name : str
    health : int
    initiative : int
    team : str
    
    def __init__(self):
        self.name = ""
        self.health = 0
        self.initiative = 0
        self.team = None
    def __init__(self,name,health,initiative,team):
        self.name = name
        self.health = health
        self.initiative = initiative
        self.team = team

In [36]:
class GroupLoop:
    group : pd.DataFrame
    name : str

    def __init__(self):
        self.group = pd.DataFrame()
        self.name = "Unnamed Group"
    def __init__(self, list_of_characters,group_name):
        self.group = pd.DataFrame(list_of_characters)
        self.name = group_name
    def set_next_group(self, next_group):
        self.next_group = next_group